In [12]:

!git clone https://github.com/MVIG-SJTU/AlphaPose.git
!cd AlphaPose
!pip install -r requirements.txt

El sistema no puede encontrar la ruta especificada.


In [15]:
!python -m venv alphapose_env
!alphapose_env\Scripts\activate

fatal: destination path 'C:\Users\malej\OneDrive\Escritorio\Cosas de Sebastian\Tesis Master\openpose' already exists and is not an empty directory.


In [ ]:
import torch
import cv2
import numpy as np
from alphapose.models import builder
from alphapose.utils.config import update_config
from alphapose.utils.presets import SimpleTransform
from detector.apis import get_detector
from alphapose.utils.transforms import get_func_heatmap_to_coord

def process_image(image_path):
    cfg = update_config('configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml')
    detector = get_detector(cfg)

    pose_model = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)
    pose_model.load_state_dict(torch.load(cfg.MODEL.PRETRAINED, map_location='cpu'))

    pose_model = pose_model.cuda()
    pose_model.eval()

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    bbox = detector(image)
    bbox = bbox[:, 0:4]

    transform = SimpleTransform(
        pose_model.input_size, pose_model.output_size,
        mean=pose_model.pixel_mean, std=pose_model.pixel_std, scale_factor=cfg.DATASET.SCALE_FACTOR,
        rotation_factor=cfg.DATASET.ROT_FACTOR, do_flip=cfg.DATASET.DO_FLIP,
        keypoints_flip=cfg.DATASET.KEYPOINTS_FLIP, keypoints_order=cfg.DATASET.KEYPOINTS_ORDER,
        keypoints_symmetry=cfg.DATASET.KEYPOINTS_SYMMETRY, aug_seq=cfg.DATASET.AUG_SEQ)

    with torch.no_grad():
        batch_data = []
        for box in bbox:
            input_data, orig_image, center, scale = transform(image, box)
            batch_data.append(input_data)
        
        batch_data = torch.stack(batch_data).cuda()
        heatmaps = pose_model(batch_data)
        coords = get_func_heatmap_to_coord(cfg.DATA_PRESET)
        preds = coords(heatmaps, batch_data)

    return preds

if __name__ == "__main__":
    preds = process_image('img.jpg')
    print(preds)


In [ ]:
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

input_size = 17 * 2
hidden_size = 128
num_layers = 2
num_classes = 10 

model = SimpleRNN(input_size, hidden_size, num_layers, num_classes)



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from rnn_model import SimpleRNN
import numpy as np

# Número de secuencias
num_sequences = 5
# Longitud de cada secuencia (número de fotogramas)
sequence_length = 10
# Número de puntos clave
num_keypoints = 17
# Número de características por punto clave (x, y)
num_features = 2


training_data = np.random.rand(num_sequences, sequence_length, num_keypoints * num_features)

labels = np.random.randint(0, 10, num_sequences)

# Convertir a tensores
inputs = torch.tensor(training_data, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.long)

# Hiperparámetros
learning_rate = 0.001
num_epochs = 100
hidden_size = 128
num_layers = 2
input_size = num_keypoints * num_features
num_classes = 10

model = SimpleRNN(input_size, hidden_size, num_layers, num_classes)
model = model.cuda() if torch.cuda.is_available() else model

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Bucle de entrenamiento
for epoch in range(num_epochs):
    model.train()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
